# Snow cover mapping of Colombian glaciers

## NDSI calculation and exporting

In [1]:
#Import Earth Engine library and initialize
import ee
ee.Authenticate(auth_mode="notebook")
ee.Initialize()

In [2]:
#Load regions glacier points and apply a buffer to define the area of interest

gpoints = ee.FeatureCollection("users/sergioingeo/glacier/colombian_glaciers") #Change this with you glacier points of interest

points = ee.Geometry.MultiPoint(gpoints.geometry().coordinates())

aoi = points.buffer(30000)

#Load landsat collections
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterBounds(aoi).filterDate("2021-01-01", "2023-12-31")

l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(aoi).filterDate("2021-01-01", "2023-12-31")

l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterBounds(aoi).filterDate("2002-01-01", "2004-12-31")

l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterBounds(aoi).filterDate("1985-01-01", "1987-12-31")

#Load elevation data
COPDEM30 = ee.ImageCollection("COPERNICUS/DEM/GLO30").select("DEM").reduce(ee.Reducer.mean()).clip(aoi)

In [3]:
#Create functions to mask clouds, and add snow index as bands

def maskClouds(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudBitMask = ee.Number(2).pow(4).int()
    #Select the QA band
    qa = image.select("QA_PIXEL")
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudBitMask).eq(0))
    
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

def addsnowindexl57(image):
    NDSI = image.normalizedDifference(["SR_B2", "SR_B5"]).rename("NDSI")
    return image.addBands(NDSI)
                          
def addsnowindexl9(image):
    NDSI = image.normalizedDifference(["SR_B3", "SR_B6"]).rename("NDSI")
    return image.addBands(NDSI)

def applyScaleFactor(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(srcImg = opticalBands, overwrite = True)

In [4]:
#Apply filters and masks

landsat9 = l9.map(applyScaleFactor).map(maskClouds).map(addsnowindexl9)

landsat8 = l8.map(applyScaleFactor).map(maskClouds).map(addsnowindexl9)

#Join landsat 9 and 8 collections
landsat89 = landsat9.merge(landsat8)

landsat7 = l7.map(applyScaleFactor).map(maskClouds).map(addsnowindexl57)

landsat5 = l5.map(applyScaleFactor).map(maskClouds).map(addsnowindexl57)

#Composite the landsat imagery and apply elevation mask

elevationmask = COPDEM30.gt(3800)

composite2022 = landsat89.median().clip(aoi).updateMask(elevationmask)

composite2003 = landsat7.median().clip(aoi).updateMask(elevationmask)

composite1986 = landsat5.median().clip(aoi).updateMask(elevationmask)


In [6]:
#Select the NDSI bands for exporting

NDSI2022 = composite2022.select("NDSI")

NDSI2003 = composite2003.select("NDSI")

NDSI1986 = composite1986.select("NDSI")

NDSILIST = [NDSI2022, NDSI2003, NDSI1986]

yearlist = [2022, 2003, 1986]

In [45]:
#Create an exporting task to export all results to Google Drive
yearindex = 0
for img in NDSILIST:
    task = ee.batch.Export.image.toDrive(
        image=img,
        region=aoi,
        description=f"NDSI_{yearlist[yearindex]}",
        scale=30,
        maxPixels=1e13,
        )
    task.start()
    yearindex += 1 

## Snow cover mapping

In [11]:
# Use the NDSI threshold of 0.55 to select snow pixels

snow_pixels_2022 = NDSI2022.gt(0.55)

snow_pixels_2022  = snow_pixels_2022.updateMask(snow_pixels_2022)

snow_pixels_2003 = NDSI2003.gt(0.55)

snow_pixels_2003  = snow_pixels_2003.updateMask(snow_pixels_2003)

snow_pixels_1986 = NDSI1986.gt(0.55)

snow_pixels_1986  = snow_pixels_1986.updateMask(snow_pixels_1986)

snowcoverlist = [snow_pixels_2022, snow_pixels_2003, snow_pixels_1986]

In [ ]:
#Create an exporting task to export all results to Google Drive
yearindex = 0
for snowcover in snowcoverlist:
    task = ee.batch.Export.image.toDrive(
        image=snowcover,
        region=aoi,
        description=f"SNOWCOVER_{yearlist[yearindex]}",
        scale=30,
        maxPixels=1e13,
        )
    task.start()
    yearindex += 1 